In [1]:
import pandas as pd
import numpy as np
import holidays

# Load datasets
all_listings = pd.read_csv("/content/drive/MyDrive/merged_data/all_listings.csv")
all_calendars = pd.read_csv("/content/drive/MyDrive/merged_data/all_calendars.csv")

In [2]:
# ==============================================
# 1. Data Cleaning for Listings (Enhanced)
# ==============================================
def clean_listings(df: pd.DataFrame) -> pd.DataFrame:
    """Clean and engineer features for the listings dataset."""
    # Fix neighborhood names manually
    def fix_neighborhood_manual(neighborhood: str) -> str:
        if pd.isna(neighborhood):
            return neighborhood
        fixes = {
            "Vieux-Qu√©bec/Cap-Blanc/Colline parlementaire": "Vieux-Québec/Cap-Blanc/Colline parlementaire",
            "Qu√©bec": "Québec",
        }
        return fixes.get(neighborhood, neighborhood)

    df['neighbourhood_cleansed'] = df['neighbourhood_cleansed'].apply(fix_neighborhood_manual)

    # 1. Description handling
    df['has_description'] = df['description'].notna().astype(int)

    # 2. Host response time - preserve original categories
    df['host_response_time'] = df['host_response_time'].fillna('no_response')

    # 3. Superhost conversion with validation
    df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0, None: 0}).fillna(0)

    # 4. Bathroom features with enhanced pattern matching
    bath_pattern = r'shared|share|communal|joint'
    df['shared_bath'] = df['bathrooms_text'].str.lower().str.contains(bath_pattern, na=False).astype(int)

    # 5. Numeric columns handling with improved outlier detection
    numeric_cols = ['accommodates', 'bathrooms', 'bedrooms', 'beds']
    for col in numeric_cols:
        # Calculate median before handling outliers
        col_median = df[col].median()
        # Fill NA with median
        df[col] = df[col].fillna(col_median)
        # Clip values between 1st and 99th percentiles
        lower = df[col].quantile(0.01)
        upper = df[col].quantile(0.99)
        df[col] = df[col].clip(lower, upper)
        # Ensure non-negative values
        df[col] = df[col].clip(lower=0)

    # 6. Enhanced logical constraints
    df['bedrooms'] = np.where(df['bedrooms'] > df['accommodates'], df['accommodates'], df['bedrooms'])
    df['bathrooms'] = np.where(df['bathrooms'] > df['bedrooms'], df['bedrooms'], df['bathrooms'])
    df['beds'] = np.where(df['beds'] > df['bathrooms']*2, df['bathrooms']*2, df['beds'])  # Assume max 2 beds per bathroom

    # 7. Advanced amenities processing
    amenities = df['amenities'].str.lower()
    df['parking'] = amenities.str.contains(r'\bparking\b', regex=True).astype(int)
    df['air_conditioning'] = amenities.str.contains(r'air conditioning| a/c\b|ac unit', regex=True).astype(int)
    df['wifi'] = amenities.str.contains(r'wifi|internet|wireless', regex=True).astype(int)

    # 8. Enhanced price handling with currency validation
    df['price'] = (
        df['price']
        .replace('[^0-9.]', '', regex=True)  # Remove non-numeric characters
        .astype(float)
    )
    # Handle missing prices using city-level median
    df['price'] = df.groupby('city')['price'].transform(
        lambda x: x.fillna(x.median())
    )
    # Outlier handling using city-specific ranges
    def city_clipper(group):
        q1 = group.quantile(0.05)
        q3 = group.quantile(0.95)
        return group.clip(q1, q3)
    df['price'] = df.groupby('city')['price'].transform(city_clipper)

    # 9. Enhanced review handling
    df['has_review'] = df['review_scores_rating'].notna().astype(int)
    df['review_scores_rating'] = df['review_scores_rating'].fillna(0)

    # 10. Instant bookable with validation
    df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0, None: 0}).fillna(0)

    # Drop unnecessary columns
    df = df.drop(columns=['amenities', 'description', 'bathrooms_text'])

    return df

# Apply cleaning to listings
cleaned_listings = clean_listings(all_listings)

In [3]:
# ==============================================
# 2. Data Cleaning for Calendars (Enhanced)
# ==============================================
def clean_calendars(df: pd.DataFrame) -> pd.DataFrame:
    """Clean and engineer features for the calendar dataset."""
    # Convert date column
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # 1. Enhanced availability conversion
    df['occupied'] = (
        df['available']
        .replace({'t': 0, 'f': 1, '': np.nan, None: np.nan})
        .astype(float)
    )
    df['occupied'] = df['occupied'].fillna(0).astype(int)

    # 2. Seasonal features with proper date handling
    df['month'] = df['date'].dt.month
    seasons = [
        (df['month'].isin([12, 1, 2]), 'Winter'),
        (df['month'].isin([3, 4, 5]), 'Spring'),
        (df['month'].isin([6, 7, 8]), 'Summer'),
        (df['month'].isin([9, 10, 11]), 'Fall')
    ]
    df['season'] = np.select([cond for cond, _ in seasons], [label for _, label in seasons])

    # 3. Weekend flag with edge case handling
    df['is_weekend'] = (df['date'].dt.dayofweek >= 5).astype(int)

    # 4. Enhanced Canadian holiday handling
    ca_holidays = holidays.CA(years=2024)
    additional_holidays = {
        '2024-04-01': "Easter Monday (Observed)",
        '2024-05-20': "Victoria Day",
        '2024-07-01': "Canada Day",
        '2024-09-02': "Labour Day",
        '2024-10-14': "Thanksgiving",
        '2024-12-25': "Christmas Day",
        '2024-12-26': "Boxing Day"
    }
    df['is_holiday'] = (
        df['date'].astype(str).isin(additional_holidays.keys())
        | df['date'].isin(ca_holidays)
    ).astype(int)
    false_positives = ['2024-03-29']
    df['is_holiday'] = np.where(
        df['date'].astype(str).isin(false_positives),
        0,
        df['is_holiday']
    )

    return df

# Apply cleaning to calendars
cleaned_calendars = clean_calendars(all_calendars)

<ipython-input-3-b10cb90862d4>:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({'t': 0, 'f': 1, '': np.nan, None: np.nan})
<ipython-input-3-b10cb90862d4>:43: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  | df['date'].isin(ca_holidays)


In [4]:
# ==============================================
# 3. Calculate occupancy_365 and import to listings file
# ==============================================
occupancy_365 = (
    cleaned_calendars.groupby('listing_id')['occupied']
    .mean()
    .reset_index()
    .rename(columns={'occupied': 'occupancy_365'})
)

cleaned_listings = pd.merge(
    cleaned_listings,
    occupancy_365,
    left_on='id',
    right_on='listing_id',
    how='left'
).drop(columns=['listing_id'])

# Fill NA with 0 (if the listing has no occupancy data)
cleaned_listings['occupancy_365'] = cleaned_listings['occupancy_365'].fillna(0)

In [5]:


# ==============================================
# 4. Save Cleaned Data
# ==============================================
cleaned_listings.to_csv("cleaned_listings_v2.csv", index=False)
cleaned_calendars.to_csv("cleaned_calendars_v2.csv", index=False)

print("Data cleaning complete!")
print(f"Listings shape: {cleaned_listings.shape}")
print(f"Calendars shape: {cleaned_calendars.shape}")

Data cleaning complete!
Listings shape: (34734, 23)
Calendars shape: (12712644, 9)


In [ ]:
# Create daily metrics for visualization
daily_metrics = cleaned_calendars.groupby(['city', 'date']).agg(
    total_listings=('listing_id', 'nunique'),
    occupied=('occupied', 'sum')
).reset_index()
daily_metrics['occupancy_rate'] = daily_metrics['occupied'] / daily_metrics['total_listings']
daily_metrics.to_csv("daily_metrics.csv", index=False)
print(f"Daily Metrics shape: {daily_metrics.shape}")